In [2]:
import pandas as pd
from tqdm import tqdm
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES


MAIN_DIMENSIONS = [
    "Abdomen",
    "Musculoskeletal",
    "Lungs",
    "Eyes",
    "Heart",
    "Arterial",
    "Brain",
    "Biochemistry",
    "Hearing",
    "BloodCells",
    "PhysicalActivity",
]

PAIRS_MAIN_DIMENSIONS = [
    [main_dim_1, main_dim_2]
    for idx_dim, main_dim_1 in enumerate(MAIN_DIMENSIONS)
    for main_dim_2 in MAIN_DIMENSIONS[idx_dim + 1 :]
]

PAIRS_SUBDIMENSIONS = [
    ["BrainMRI", "BrainCognitive"],
    ["EyesOCT", "EyesFundus"],
    ["HeartECG", "HeartMRI"],
    ["AbdomenLiver", "AbdomenPancreas"],
    ["BiochemistryBlood", "BiochemistryUrine"],
    ["MusculoskeletalScalars", "MusculoskeletalFullBody"],
    ["MusculoskeletalScalars", "MusculoskeletalSpine"],
    ["MusculoskeletalScalars", "MusculoskeletalHips"],
    ["MusculoskeletalScalars", "MusculoskeletalKnees"],
    ["MusculoskeletalFullBody", "MusculoskeletalSpine"],
    ["MusculoskeletalFullBody", "MusculoskeletalHips"],
    ["MusculoskeletalFullBody", "MusculoskeletalKnees"],
    ["MusculoskeletalSpine", "MusculoskeletalHips"],
    ["MusculoskeletalSpine", "MusculoskeletalKnees"],
    ["MusculoskeletalHips", "MusculoskeletalKnees"],
]
DIMENSIONS_TO_EXCLUDE = {
    "set": [],
    "set_instances01": [],
    "set_instances1.5x": [],
    "set_instances23": [],
    "Abdomen": ["AbdomenLiver", "AbdomenPancreas"],
    "AbdomenLiver": ["Abdomen"],
    "AbdomenPancreas": ["Abdomen"],
    "Arterial": ["ArterialCarotids", "ArterialPulseWaveAnalysis"],
    "ArterialCarotids": ["Arterial"],
    "ArterialPulseWaveAnalysis": ["Arterial"],
    "Biochemistry": ["BiochemistryBlood", "BiochemistryUrine"],
    "BiochemistryBlood": ["Biochemistry"],
    "BiochemistryUrine": ["Biochemistry"],
    "Brain": ["BrainCognitive", "BrainMRI"],
    "BrainCognitive": ["Brain"],
    "BrainMRI": ["Brain"],
    "Eyes": ["EyesAll", "EyesFundus", "EyesOCT"],
    "EyesAll": ["Eyes"],
    "EyesFundus": ["Eyes"],
    "EyesOCT": ["Eyes"],
    "Hearing": [],
    "Heart": ["HeartECG", "HeartMRI"],
    "HeartECG": ["Heart"],
    "HeartMRI": ["Heart"],
    "BloodCells": [],
    "Lungs": [],
    "Musculoskeletal": [
        "MusculoskeletalFullBody",
        "MusculoskeletalHips",
        "MusculoskeletalKnees",
        "MusculoskeletalScalars",
        "MusculoskeletalSpine",
    ],
    "MusculoskeletalFullBody": ["Musculoskeletal"],
    "MusculoskeletalHips": ["Musculoskeletal"],
    "MusculoskeletalKnees": ["Musculoskeletal"],
    "MusculoskeletalScalars": ["Musculoskeletal"],
    "MusculoskeletalSpine": ["Musculoskeletal"],
    "PhysicalActivity": [],
}

In [24]:
correlations_raw = pd.read_feather(f"../../data/xwas/multivariate_correlations/correlations/correlations.feather").set_index(
    ["dimension_1", "dimension_2", "category"]
)
correlations_raw.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations_raw.columns.tolist())), names=["algorithm", "correlation_type"]
)
every_category = correlations_raw.index.get_level_values("category").drop_duplicates()

list_indexes = []
for dimension in DIMENSIONS + ["MainDimensions", "SubDimensions"]:
    for category in every_category:
        list_indexes.append([dimension, category])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["dimension", "category"])

list_columns = []
for algorithm in ["elastic_net", "light_gbm", "neural_network"]:
    for correlation_type in ["pearson", "spearman"]:
        for observation in ["mean", "std"]:
            list_columns.append([algorithm, correlation_type, observation])
columns = pd.MultiIndex.from_tuples(list_columns, names=["algorithm", "correlation_type", "observation"])

averages_correlations = pd.DataFrame(None, index=indexes, columns=columns)

In [27]:
for algorithm in ["elastic_net", "light_gbm", "neural_network"]:
    for correlation_type in ["pearson", "spearman"]:
        correlations = correlations_raw[algorithm, correlation_type].swaplevel().swaplevel(i=0, j=1)

        for category in every_category:
            correlations_category = correlations.loc[category]

            averages_correlations.loc[
                ("MainDimensions", category), (algorithm, correlation_type, "mean")
            ] = correlations_category.loc[PAIRS_MAIN_DIMENSIONS].mean()
            averages_correlations.loc[
                ("MainDimensions", category), (algorithm, correlation_type, "std")
            ] = correlations_category.loc[PAIRS_MAIN_DIMENSIONS].std()

            averages_correlations.loc[
                ("SubDimensions", category), (algorithm, correlation_type, "mean")
            ] = correlations_category.loc[PAIRS_SUBDIMENSIONS].mean()
            averages_correlations.loc[
                ("SubDimensions", category), (algorithm, correlation_type, "std")
            ] = correlations_category.loc[PAIRS_SUBDIMENSIONS].std()

            for dimension in DIMENSIONS:
                correlations_independant = correlations_category.loc[dimension].drop(
                    index=([dimension] + DIMENSIONS_TO_EXCLUDE[dimension])
                )

                averages_correlations.loc[
                    (dimension, category), (algorithm, correlation_type, "mean")
                ] = correlations_independant.mean()
                averages_correlations.loc[
                    (dimension, category), (algorithm, correlation_type, "std")
                ] = correlations_independant.std()

averages_correlations.columns = map(str, averages_correlations.columns.tolist())
averages_correlations.reset_index().to_feather("data/xwas/multivariate_correlations/averages_correlations.feather")

('elastic_net', 'pearson', 'mean')  \
dimension     category                                                              
Abdomen       Alcohol                                                    0.589633   
              Diet                                                       0.540979   
              Education                                                  0.684668   
              ElectronicDevices                                          0.605466   
              Employment                                                 0.673773   
...                                                                           ...   
SubDimensions CognitiveProspectiveMemory                                 0.658322   
              CognitiveReactionTime                                      0.755864   
              CognitiveSymbolDigitSubstitution                           0.672218   
              CognitiveTowerRearranging                                  0.667571   
              CognitiveTrailMaking                                        0.68661   

                                               ('elastic_net', 'pearson', 'std')  \
dimension     category                                                             
Abdomen       Alcohol                                                   0.135176   
              Diet                                                      0.159024   
              Education                                                 0.232164   
              ElectronicDevices                                         0.239317   
              Employment                                                0.185456   
...                                                                          ...   
SubDimensions CognitiveProspectiveMemory                                0.225951   
              CognitiveReactionTime                                     0.114107   
              CognitiveSymbolDigitSubstitution                          0.143121   
              CognitiveTowerRearranging                                 0.145429   
              CognitiveTrailMaking                                      0.139754   

                                               ('elastic_net', 'spearman', 'mean')  \
dimension     category                                                               
Abdomen       Alcohol                                                     0.516687   
              Diet                                                        0.303607   
              Education                                                   0.493342   
              ElectronicDevices                                           0.479922   
              Employment                                                  0.562271   
...                                                                            ...   
SubDimensions CognitiveProspectiveMemory                                  0.597765   
              CognitiveReactionTime                                       0.645017   
              CognitiveSymbolDigitSubstitution                            0.602311   
              CognitiveTowerRearranging                                   0.562724   
              CognitiveTrailMaking                                         0.59987   

                                               ('elastic_net', 'spearman', 'std')  \
dimension     category                                                              
Abdomen       Alcohol                                                    0.156497   
              Diet                                                        0.14938   
              Education                                                  0.154985   
              ElectronicDevices                                          0.159897   
              Employment                                                 0.155138   
...                                                                           ...   
SubDimensions CognitiveProspectiveMemory         